In [16]:
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score
import math

1) Загрузите данные из файла data-logistic.csv. Это двумерная выборка, целевая переменная на которой принимает значения -1 или 1.

In [17]:
data = pd.read_csv('csv_dataframes/data-logistic.csv',header = None)
X = data[[1,2]]
y = data[0]

2) Убедитесь, что выше выписаны правильные формулы для градиентного спуска. Обратите внимание, что мы используем полноценный градиентный спуск, а не его стохастический вариант!

3) Реализуйте градиентный спуск для обычной и L2-регуляризованной (с коэффициентом регуляризации 10) логистической регрессии. Используйте длину шага k=0.1. В качестве начального приближения используйте вектор (0, 0).

4) Запустите градиентный спуск и доведите до сходимости (евклидово расстояние между векторами весов на соседних итерациях должно быть не больше 1e-5). Рекомендуется ограничить сверху число итераций десятью тысячами.

In [31]:
#step_len ~ k
#reg_coef ~ C

def fw1(w1, w2, y, X, step_len, reg_coef):
    l = len(y)
    S = 0
    for i in range(0, l):
        S += y[i] * X[1][i] * (1.0 - 1.0 / (1.0 + math.exp(-y[i] * (w1*X[1][i] + w2*X[2][i]))))

    return w1 + ((step_len / l) * S) - step_len * reg_coef * w1

def fw2(w1, w2, y, X, step_len, reg_coef):
    l = len(y)
    S = 0
    for i in range(0, l):
        S += y[i] * X[2][i] * (1.0 - 1.0 / (1.0 + math.exp(-y[i] * (w1*X[1][i] + w2*X[2][i]))))

    return w2 + ((step_len / l) * S) - step_len * reg_coef * w2

def grad(y, X, step_len, err, i_max, reg_coef, w1=0.0, w2=0.0):
    i = 0
    i_max = i_max
    err = err
    step_len = step_len
    reg_coef = reg_coef
    w1_new, w2_new = w1, w2

    while True:
        i += 1
        w1_new, w2_new = fw1(w1, w2, y, X, step_len, reg_coef), fw2(w1, w2, y, X, step_len, reg_coef)
        e = math.sqrt((w1_new - w1) ** 2 + (w2_new - w2) ** 2)

        if i >= i_max or e <= err:
            break
        else:
            w1, w2 = w1_new, w2_new

    return [w1_new, w2_new]

w1, w2 = grad(y, X, step_len=0.1, err=1e-5, i_max=10000, reg_coef=0.0)
reg_w1, reg_w2 = grad(y, X, step_len=0.1, err=1e-5, i_max=10000, reg_coef=10.0)
print(w1,w2)
print(reg_w1,reg_w2)

0.2878116204717764 0.09198330215925439
0.028558754546234223 0.02478013724973556


5) Какое значение принимает AUC-ROC на обучении без регуляризации и при ее использовании? Эти величины будут ответом на задание. В качестве ответа приведите два числа через пробел. Обратите внимание, что на вход функции roc_auc_score нужно подавать оценки вероятностей, подсчитанные обученным алгоритмом. Для этого воспользуйтесь сигмоидной функцией: 
$$a(x) = \frac{1}{1 + \exp(-w_1 x_1 - w_2 x_2)} $$

In [32]:
def a(X, w1, w2):
    return 1.0 / (1.0 + math.exp(-w1 * X[1] - w2 * X[2]))

#axis = 1 - по строчкам, x - вектор
y_score = X.apply(lambda x: a(x, w1, w2), axis=1)
y_reg_score = X.apply(lambda x: a(x, reg_w1, reg_w2), axis=1)

#это именно площадь под кривой auc
auc = roc_auc_score(y, y_score) 
reg_auc = roc_auc_score(y, y_reg_score)
print(round(auc,3),round(reg_auc,3))

0.927 0.936


6) Попробуйте поменять длину шага. Будет ли сходиться алгоритм,
если делать более длинные шаги? Как меняется число итераций
при уменьшении длины шага?

7) Попробуйте менять начальное приближение. Влияет ли оно на чтонибудь?